# Query Rightmove with LangChain | OpenAI | Python | bs4

In [22]:
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from langchain.prompts import PromptTemplate    

import statistics

import requests
from bs4 import BeautifulSoup

In [11]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [ ]:
# Provide a URL of a list of properties to analyse

In [13]:
url='https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E116&maxBedrooms=3&minBedrooms=3&maxPrice=325000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='

## The Tools

In [14]:
@tool
def get_median_price(url=url) -> str:
    """Gets the median house price from the supplied Rightmove URL"""

    # Get the page

    headers = { 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0' }
    response = requests.get(url, headers=headers)

    # Scrape the content

    soup = BeautifulSoup(response.text, "html.parser")

    props = soup.find_all("div", class_="l-searchResult is-list")

    all_prices = []
    for i in range(len(props)):
        prop = props[i]
        price = (
            prop.find("div", class_="propertyCard-priceValue")
            .get_text()
            .replace(",","")
            .strip("£")
            .strip()
        )
        all_prices.append(int(price))
        
    res = round(statistics.median(all_prices),2)
    
    return (str(res))

In [15]:
@tool
def get_mean_price(url=url) -> str:
    """Gets the mean house price from the supplied Rightmove URL"""

    # Get the page

    headers = { 'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0' }
    response = requests.get(url, headers=headers)

    # Scrape the content

    soup = BeautifulSoup(response.text, "html.parser")

    apartments = soup.find_all("div", class_="l-searchResult is-list")

    all_prices = []
    for i in range(len(apartments)):
        apartment_no = apartments[i]
        price = (
            apartment_no.find("div", class_="propertyCard-priceValue")
            .get_text()
            .replace(",","")
            .strip("£")
            .strip()
        )
        all_prices.append(int(price))
        
    res = round(statistics.mean(all_prices),2)
    
    return (str(res))

In [16]:
get_median_price(url)

'290000'

In [17]:
get_mean_price(url)

'287772.22'

## Use Agent with Prompt

In [323]:
# Create the agent to use the tool

# List the tools that AI can use
tools = [get_median_price, get_mean_price]

# Set up the Agent, give it the tools
agent = initialize_agent(tools,
                         llm,
                         agent="chat-zero-shot-react-description", 
                         verbose=True)
                         max_iterations=3)

# Create the prompt from a prompt template, with 1 input variable
prompt = PromptTemplate(input_variables = ['calctype'],                            
          template = '''
            You have been given access to search data from Rightmove. 
            Please calculate the {calctype} based on the response using the {calctype} tool.         
           '''                                                                      
        )


# Run the agent with the provided input variable
result = agent.run(prompt.format_prompt(calctype="mean"))
print(result)



> Entering new AgentExecutor chain...
Thought: I need to use the `get_mean_price` tool to calculate the mean of the house prices from the given Rightmove URL.
Action:
```
{
  "action": "get_mean_price",
  "action_input": "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E116&maxBedrooms=3&minBedrooms=3&maxPrice=325000&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
}
```

Observation: 287772.22
Thought:I have successfully retrieved the mean house price from the given Rightmove URL.
Final Answer: The mean house price from the given Rightmove URL is 287772.22.

> Finished chain.
The mean house price from the given Rightmove URL is 287772.22.
